<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/sandboxes/BB/analysis/quac_triviaqa_text_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metrics: Text Evaluation

### Imports/Setup

In [1]:
!pip install -q transformers

     |████████████████████████████████| 5.5 MB 18.5 MB/s 
     |████████████████████████████████| 182 kB 54.9 MB/s 
     |████████████████████████████████| 7.6 MB 44.9 MB/s 


In [2]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 31.5 MB/s 


In [3]:
from os import listdir
from os.path import isfile, join

import csv
import json
import pprint

import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch

from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option("display.max_colwidth", None)

In [4]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
evaluation_root = "/content/drive/MyDrive/w266 NLP Final Project/Evaluation/"
filename = join(evaluation_root, "evaluation_database.json")

# Read JSON into dataframe
evaluation_df = pd.read_json(filename)

### Clean up evaluation dataframe to only include columns needed for text analysis

In [6]:
# Clean up evaluation db df to only include columns needed for text analysis
clean_eval_df = evaluation_df[
    [
        "nickname",
        "base_model",
        "trained_on",
        "tested_on",
        "hyperparameter",
        "target",
        "prediction",
        "bleu",
        "rougeL",
        "meteor",
        "bertscore-f1",
        "use",
    ]
]

clean_eval_df

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
0,bart_nq_nq,bart,nq,nq,{'defaults': True},what was the real name of saudi arabia,what is the name of the kingdom of saudi arabia,0.000000,0.666667,0.623306,0.915830,0.794086
1,bart_nq_nq,bart,nq,nq,{'defaults': True},whats the most liked picture on instagram 2018,what is the most liked picture on instagram,0.680375,0.750000,0.864796,0.932259,0.884909
2,bart_nq_nq,bart,nq,nq,{'defaults': True},where does the movie proof of life take place,where does alice go in the new movie,0.000000,0.470588,0.354635,0.835192,0.517866
3,bart_nq_nq,bart,nq,nq,{'defaults': True},where is net profit on the balance sheet,where does net profit come from in a financial statement,0.000000,0.333333,0.311653,0.856130,0.728012
4,bart_nq_nq,bart,nq,nq,{'defaults': True},when was fingerprinting first used by the police,when was fingerprint technology first used in the us,0.000000,0.588235,0.694444,0.876353,0.704206
...,...,...,...,...,...,...,...,...,...,...,...,...
286285,T5_amalgam_triviaqa,T5,amalgam,triviaqa,{'defaults': True},Which ex British daily newspaper was first published in 1968 and featured the front page headline 'Second Spy Inside GCHQ'? It cost 18 pence and was owned by Eddy Shah a Manchester based businessman.,What is the name of the city of Manchester?,0.000000,0.093023,0.058309,0.679618,0.322748
286286,T5_amalgam_triviaqa,T5,amalgam,triviaqa,{'defaults': True},Which cartoon character lived in Bunkerton Castle?,What was the name of Lord Marmaduke of Bunkerton?,0.000000,0.125000,0.121951,0.794428,0.515345
286287,T5_amalgam_triviaqa,T5,amalgam,triviaqa,{'defaults': True},In which prison was 'Amy' born in the novel 'Little Dorrit'?,What is the name of the prison in Charles Dickens' novel Little Dorrit?,0.000000,0.416667,0.248227,0.821790,0.593840
286288,T5_amalgam_triviaqa,T5,amalgam,triviaqa,{'defaults': True},The Sign Of Four was a detective story written by which author?,Who wrote the novels 'The Sign of Four' and 'A Study in Scarlet'?,0.000000,0.400000,0.381426,0.796076,0.585255


In [ ]:
clean_eval_df.describe()

,bleu,rougeL,meteor,bertscore-f1,use
count,286290.000000,286290.000000,286290.000000,286290.000000,286290.000000
mean,0.048842,0.295583,0.279015,0.789801,0.483113
std,0.158423,0.233494,0.234177,0.082735,0.246386
min,0.000000,0.000000,0.000000,0.521592,-0.147977
25%,0.000000,0.125000,0.097087,0.729552,0.291765
50%,0.000000,0.250000,0.200000,0.784537,0.487792
75%,0.000000,0.428571,0.408791,0.845546,0.671069
max,1.000000,1.000000,0.999937,1.000000,1.000000


### Find the overall averages for each prediction set and metric

In [ ]:
# Find the overall averages for each prediction set and metric
eval_means_df = clean_eval_df.groupby(["nickname"]).mean()

# Reset index so able to groupby and sort below to find top scoring prediction sets for each metric
eval_means_df = eval_means_df.reset_index("nickname")

eval_means_df = eval_means_df[["nickname", "bleu", "rougeL", "meteor", "bertscore-f1", "use"]]
eval_means_df

,nickname,bleu,rougeL,meteor,bertscore-f1,use
0,T5_amalgam_nq,0.169746,0.527758,0.497871,0.858036,0.719863
1,T5_amalgam_quac,0.053671,0.249039,0.262385,0.778060,0.378001
2,T5_amalgam_squad,0.142885,0.470772,0.469798,0.867048,0.656399
3,T5_amalgam_triviaqa,0.066390,0.359130,0.337926,0.822766,0.601577
4,T5_nq_nq,0.172979,0.527675,0.497261,0.859476,0.718928
5,T5_nq_quac,0.001471,0.151900,0.111625,0.696885,0.287737
6,T5_nq_squad,0.017446,0.346182,0.273460,0.791693,0.554671
7,T5_nq_triviaqa,0.007450,0.278408,0.189823,0.766649,0.518441
8,T5_quac_nq,0.000000,0.068525,0.040712,0.662784,0.180669
9,T5_quac_quac,0.023456,0.173453,0.189914,0.756046,0.303081


### Create BART and T5 dataframes to focus on

In [ ]:
# Create BART dataframe
bart_df = clean_eval_df[clean_eval_df["base_model"] == "bart"]

In [ ]:
# Create T5 dataframe
t5_df = clean_eval_df[clean_eval_df["base_model"] == "T5"]

## Model Evaluation

### Examples the models struggle with

Models Tested on QuAC

In [7]:
quac_mods = clean_eval_df[
    (clean_eval_df["tested_on"] == "quac")
]

quac_mods = quac_mods.groupby(["base_model", "trained_on","tested_on"]).mean().sort_values(by="use", ascending=False)

# Multiply scores by 100
quac_mods = quac_mods[["bleu", "rougeL", "meteor", "bertscore-f1", "use"]].mul(100).round(2)

quac_mods

bleu  rougeL  meteor  bertscore-f1    use
base_model trained_on tested_on                                           
bart       quac       quac       5.72   28.27   28.81         79.28  41.88
T5         amalgam    quac       5.37   24.90   26.24         77.81  37.80
bart       amalgam    quac       2.74   23.37   25.02         77.05  36.38
T5         squad      quac       0.71   18.52   20.80         75.37  32.05
bart       squad      quac       0.71   18.42   20.59         75.40  31.85
T5         quac       quac       2.35   17.35   18.99         75.60  30.31
           nq         quac       0.15   15.19   11.16         69.69  28.77
bart       nq         quac       0.11   14.96   11.26         69.72  28.66
T5         triviaqa   quac       0.17   11.06   16.06         71.56  24.46
bart       triviaqa   quac       0.15   11.36   16.30         70.93  24.32

In [ ]:
# Convert to Latex
print(quac_mods.to_latex(index=False))

\begin{tabular}{rrrrr}
\toprule
 bleu &  rougeL &  meteor &  bertscore-f1 &   use \\
 5.72 &   28.27 &   28.81 &         79.28 & 41.88 \\
\midrule
 5.37 &   24.90 &   26.24 &         77.81 & 37.80 \\
 2.74 &   23.37 &   25.02 &         77.05 & 36.38 \\
 0.71 &   18.52 &   20.80 &         75.37 & 32.05 \\
 0.71 &   18.42 &   20.59 &         75.40 & 31.85 \\
 2.35 &   17.35 &   18.99 &         75.60 & 30.31 \\
 0.15 &   15.19 &   11.16 &         69.69 & 28.77 \\
 0.11 &   14.96 &   11.26 &         69.72 & 28.66 \\
 0.17 &   11.06 &   16.06 &         71.56 & 24.46 \\
 0.15 &   11.36 &   16.30 &         70.93 & 24.32 \\
\bottomrule
\end{tabular}



Best Model Trained and Tested on QuAC

In [ ]:
# Find predictions BART trained on QuAC 
# and tested on QuAC

bart_quac = clean_eval_df[
    (clean_eval_df["trained_on"] == "quac")
    & (clean_eval_df["base_model"] == "bart")
    # & (clean_eval_df["bertscore-f1"] <= 0.73)  # top 25th percentile bertscore
    # & (clean_eval_df["use"] <= 0.3)   # bottom 25th percentile use
    & (clean_eval_df["tested_on"] == "quac")
]

bart_quac = bart_quac.sort_values(by="use", ascending=False)

bart_quac

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
35476,bart_quac_quac,bart,quac,quac,{'defaults': True},what did he do as foreign secretary?,What did he do as Foreign Secretary?,0.365555,1.0,0.999023,1.000000,1.000000
33444,bart_quac_quac,bart,quac,quac,{'defaults': True},Who were her parents?,Who were her parents?,1.000000,1.0,0.996000,1.000000,1.000000
32658,bart_quac_quac,bart,quac,quac,{'defaults': True},What was the show about?,What was the show about?,1.000000,1.0,0.997685,1.000000,1.000000
35084,bart_quac_quac,bart,quac,quac,{'defaults': True},"What is ""Born into This?""",What is Born into This?,0.432982,1.0,0.754986,0.917809,1.000000
36443,bart_quac_quac,bart,quac,quac,{'defaults': True},Who were her parents?,Who were her parents?,1.000000,1.0,0.996000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
31116,bart_quac_quac,bart,quac,quac,{'defaults': True},And after the Charter?,Are there any other interesting aspects about this article?,0.000000,0.0,0.090909,0.689085,-0.035226
36737,bart_quac_quac,bart,quac,quac,{'defaults': True},Do the people care that this is happening?,What groups did he work with?,0.000000,0.0,0.113636,0.710494,-0.048853
32740,bart_quac_quac,bart,quac,quac,{'defaults': True},any intresting information?,When did Chuck Berry return to Chess?,0.000000,0.0,0.113636,0.715174,-0.065768
35750,bart_quac_quac,bart,quac,quac,{'defaults': True},Were the essays well received?,What did he do after that?,0.000000,0.0,0.081967,0.699852,-0.065921


In [ ]:
bart_quac_pred_ct = bart_quac.groupby(["prediction"]).count().sort_values(by="use", ascending=False).reset_index()

bart_quac_pred_ct = bart_quac_pred_ct[["prediction", "nickname"]].head()

bart_quac_pred_ct

,prediction,nickname
0,Are there any other interesting aspects about this article?,724
1,Did he win any awards?,140
2,What did he do after that?,119
3,Did they go on tour?,84
4,Where did he go to school?,79


In [ ]:
# Convert to Latex
print(bart_quac_pred_ct.to_latex(index=False))

\begin{tabular}{lr}
\toprule
                                                 prediction &  nickname \\
\midrule
Are there any other interesting aspects about this article? &       724 \\
                                     Did he win any awards? &       140 \\
                                 What did he do after that? &       119 \\
                                       Did they go on tour? &        84 \\
                                 Where did he go to school? &        79 \\
\bottomrule
\end{tabular}



All models trained and tested on QuAC

In [25]:
# Find predictions BART trained on QuAC 
# and tested on QuAC

mods_quac = clean_eval_df[
    (clean_eval_df["trained_on"] == "quac")
    & (clean_eval_df["tested_on"] == "quac")
]

mods_quac = mods_quac.sort_values(by="use", ascending=False)

mods_quac

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
35476,bart_quac_quac,bart,quac,quac,{'defaults': True},what did he do as foreign secretary?,What did he do as Foreign Secretary?,0.365555,1.0,0.999023,1.000000,1.000000
178622,T5_quac_quac,T5,quac,quac,{'defaults': True},What did they do after the tour,What did they do after the tour?,0.840896,1.0,0.984478,0.949186,1.000000
178910,T5_quac_quac,T5,quac,quac,{'defaults': True},what was her first film?,What was her first film?,0.759836,1.0,0.997685,1.000000,1.000000
36544,bart_quac_quac,bart,quac,quac,{'defaults': True},what happened in 1993?,What happened in 1993?,0.668740,1.0,0.996000,1.000000,1.000000
36443,bart_quac_quac,bart,quac,quac,{'defaults': True},Who were her parents?,Who were her parents?,1.000000,1.0,0.996000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
32740,bart_quac_quac,bart,quac,quac,{'defaults': True},any intresting information?,When did Chuck Berry return to Chess?,0.000000,0.0,0.113636,0.715174,-0.065768
178758,T5_quac_quac,T5,quac,quac,{'defaults': True},When did the coalition form?,Are there any other interesting aspects about this article?,0.000000,0.0,0.078125,0.669528,-0.065841
178806,T5_quac_quac,T5,quac,quac,{'defaults': True},So the AI is like getting information from a book without really knowing it,What did he do?,0.000000,0.0,0.000000,0.630591,-0.065853
35750,bart_quac_quac,bart,quac,quac,{'defaults': True},Were the essays well received?,What did he do after that?,0.000000,0.0,0.081967,0.699852,-0.065921


In [26]:
low_mods_quac = mods_quac.tail(20)
low_mods_quac = low_mods_quac[["target", "prediction"]]
low_mods_quac

,target,prediction
178719,When in 2002 did this happen?,Are there any other interesting aspects about this article?
177407,Was this an honest mistake by the media?,What did he do after his death?
175571,What can you tell me about fundamental elements of southtown?,Did they release any singles?
32435,How did imbreedng hurt the horses?,Are there any other interesting aspects about this article?
178048,Are they causing problems that would make the policy makers treat them so badly?,What did he do in Tanzania?
178514,Where do Sherpa people pray ?,Are there any other interesting aspects about this article?
176959,When was Kevin accused?,Are there any other interesting aspects about this article?
36570,Is there anything else you found interesting?,When did he become prime minister?
31116,And after the Charter?,Are there any other interesting aspects about this article?
179204,WHen did she first learn how to dance?,Are there any other interesting aspects about this article?


In [27]:
# Convert to Latex
print(low_mods_quac.to_latex(index=False))

\begin{tabular}{ll}
\toprule
                                                                          target &                                                  prediction \\
\midrule
                                                   When in 2002 did this happen? & Are there any other interesting aspects about this article? \\
                                        Was this an honest mistake by the media? &                             What did he do after his death? \\
                   What can you tell me about fundamental elements of southtown? &                               Did they release any singles? \\
                                              How did imbreedng hurt the horses? & Are there any other interesting aspects about this article? \\
Are they causing problems that would make the policy makers treat them so badly? &                                 What did he do in Tanzania? \\
                                                   Where do Sherpa people pray ? & Are

In [ ]:
quac_pred_ct = mods_quac.groupby(["prediction"]).count().sort_values(by="use", ascending=False).reset_index()

quac_pred_ct = quac_pred_ct[["prediction", "nickname"]].head()

quac_pred_ct

,prediction,nickname
0,Are there any other interesting aspects about this article?,1942
1,Did he win any awards?,1095
2,What did he do after his career?,520
3,Did he have any other albums?,264
4,Did they release any singles?,233


In [ ]:
# Convert to Latex
print(quac_pred_ct["prediction"].to_latex(index=False))

\begin{tabular}{l}
\toprule
                                                 prediction \\
\midrule
Are there any other interesting aspects about this article? \\
                                     Did he win any awards? \\
                           What did he do after his career? \\
                              Did he have any other albums? \\
                              Did they release any singles? \\
\bottomrule
\end{tabular}



Trained and Tested on QuAC

In [ ]:
# Find predictions with low semantic metric scores when trained on QuAC 
# and tested on QuAC

quac_low = clean_eval_df[
    (clean_eval_df["trained_on"] == "quac")
    & (clean_eval_df["base_model"] == "T5")
    & (clean_eval_df["bertscore-f1"] <= 0.73)  # top 25th percentile bertscore
    & (clean_eval_df["use"] <= 0.3)   # bottom 25th percentile use
    & (clean_eval_df["tested_on"] == "quac")
]

quac_low = quac_low.sort_values(by="use", ascending=False)

quac_low

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
176370,T5_quac_quac,T5,quac,quac,{'defaults': True},What else is she known for?,Are there any other interesting aspects about this article?,0.0,0.000,0.068493,0.713666,0.299970
179941,T5_quac_quac,T5,quac,quac,{'defaults': True},what was their scoring average during the playoffs?,Did he win any awards?,0.0,0.000,0.057471,0.718418,0.299201
174288,T5_quac_quac,T5,quac,quac,{'defaults': True},What made him leave the team,Did he win any awards?,0.0,0.000,0.000000,0.691462,0.299080
175465,T5_quac_quac,T5,quac,quac,{'defaults': True},What did people not like about it?,Are there any other interesting aspects about this article?,0.0,0.125,0.121951,0.703159,0.298942
177160,T5_quac_quac,T5,quac,quac,{'defaults': True},What was the name of it?,Did she have any other films?,0.0,0.000,0.071429,0.699655,0.298933
...,...,...,...,...,...,...,...,...,...,...,...,...
178261,T5_quac_quac,T5,quac,quac,{'defaults': True},where was his funeral?,Are there any other interesting aspects about this article?,0.0,0.000,0.090909,0.656769,-0.036675
178502,T5_quac_quac,T5,quac,quac,{'defaults': True},When did Frank Sinatra die?,Are there any other interesting aspects about this article?,0.0,0.000,0.078125,0.627769,-0.056397
177157,T5_quac_quac,T5,quac,quac,{'defaults': True},when was his funeral?,Are there any other interesting aspects about this article?,0.0,0.000,0.090909,0.651531,-0.058868
178758,T5_quac_quac,T5,quac,quac,{'defaults': True},When did the coalition form?,Are there any other interesting aspects about this article?,0.0,0.000,0.078125,0.669528,-0.065841


In [ ]:
# Show examples where it looks like a "default" question is being generated
spec_ex = quac_low[
    (quac_low["prediction"] == "Are there any other interesting aspects about this article?")
]

spec_ex = spec_ex[["nickname", "target", "prediction", "bleu", "rougeL", "meteor", "bertscore-f1", "use"]]

spec_ex

,nickname,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
176370,T5_quac_quac,What else is she known for?,Are there any other interesting aspects about this article?,0.0,0.000000,0.068493,0.713666,0.299970
175465,T5_quac_quac,What did people not like about it?,Are there any other interesting aspects about this article?,0.0,0.125000,0.121951,0.703159,0.298942
176348,T5_quac_quac,Name another song you read about,Are there any other interesting aspects about this article?,0.0,0.133333,0.078125,0.681636,0.298399
174804,T5_quac_quac,Has he worked with anyone notable?,Are there any other interesting aspects about this article?,0.0,0.000000,0.068493,0.727908,0.296238
176672,T5_quac_quac,What else can you tell me?,Are there any other interesting aspects about this article?,0.0,0.000000,0.068493,0.713406,0.294384
...,...,...,...,...,...,...,...,...
175854,T5_quac_quac,How do Pasteur and fermentation go together?,Are there any other interesting aspects about this article?,0.0,0.000000,0.060976,0.647494,-0.036667
178261,T5_quac_quac,where was his funeral?,Are there any other interesting aspects about this article?,0.0,0.000000,0.090909,0.656769,-0.036675
178502,T5_quac_quac,When did Frank Sinatra die?,Are there any other interesting aspects about this article?,0.0,0.000000,0.078125,0.627769,-0.056397
177157,T5_quac_quac,when was his funeral?,Are there any other interesting aspects about this article?,0.0,0.000000,0.090909,0.651531,-0.058868


In [ ]:
gen_q = spec_ex.tail()

gen_q

,nickname,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
175854,T5_quac_quac,How do Pasteur and fermentation go together?,Are there any other interesting aspects about this article?,0.0,0.0,0.060976,0.647494,-0.036667
178261,T5_quac_quac,where was his funeral?,Are there any other interesting aspects about this article?,0.0,0.0,0.090909,0.656769,-0.036675
178502,T5_quac_quac,When did Frank Sinatra die?,Are there any other interesting aspects about this article?,0.0,0.0,0.078125,0.627769,-0.056397
177157,T5_quac_quac,when was his funeral?,Are there any other interesting aspects about this article?,0.0,0.0,0.090909,0.651531,-0.058868
178758,T5_quac_quac,When did the coalition form?,Are there any other interesting aspects about this article?,0.0,0.0,0.078125,0.669528,-0.065841


In [ ]:
# Convert to Latex
print(gen_q.to_latex(index=False))

\begin{tabular}{lllrrrrr}
\toprule
    nickname &                                       target &                                                  prediction &  bleu &  rougeL &   meteor &  bertscore-f1 &       use \\
\midrule
T5\_quac\_quac & How do Pasteur and fermentation go together? & Are there any other interesting aspects about this article? &   0.0 &     0.0 & 0.060976 &      0.647494 & -0.036667 \\
T5\_quac\_quac &                       where was his funeral? & Are there any other interesting aspects about this article? &   0.0 &     0.0 & 0.090909 &      0.656769 & -0.036675 \\
T5\_quac\_quac &                  When did Frank Sinatra die? & Are there any other interesting aspects about this article? &   0.0 &     0.0 & 0.078125 &      0.627769 & -0.056397 \\
T5\_quac\_quac &                        when was his funeral? & Are there any other interesting aspects about this article? &   0.0 &     0.0 & 0.090909 &      0.651531 & -0.058868 \\
T5\_quac\_quac &                 When 

In [ ]:
spec_ex.groupby(["nickname"]).count()

,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
nickname,,,,,,,
T5_quac_quac,909,909,909,909,909,909,909


Trained on QuAC Tested on NQ/SQuAD

In [ ]:
# Find predictions with low semantic metric scores when trained on QuAC 
# and tested on QuAC

quac_low = clean_eval_df[
    (clean_eval_df["trained_on"] == "quac")
    & (clean_eval_df["base_model"] == "T5")
    & (clean_eval_df["bertscore-f1"] <= 0.73)  # top 25th percentile bertscore
    & (clean_eval_df["use"] <= 0.3)   # bottom 25th percentile use
    & (clean_eval_df["tested_on"] != "quac")
    # & ((clean_eval_df["tested_on"] == "nq") | (clean_eval_df["tested_on"] == "squad"))
]

quac_low = quac_low.sort_values(by="use", ascending=False)

quac_low

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
188648,T5_quac_squad,T5,quac,squad,{'defaults': True},What is the minimum amount of time before a bill can go into law?,What did he do in the Senate?,0.0,0.190476,0.104895,0.727119,0.299981
195614,T5_quac_triviaqa,T5,quac,triviaqa,{'defaults': True},"Who directed the Noughties films One Day in September, The Last King of Scotland and Touching the Void?",What did he do in the film?,0.0,0.160000,0.106383,0.728082,0.299931
197825,T5_quac_triviaqa,T5,quac,triviaqa,{'defaults': True},Which Englsih football club play home matches at the Sports Direct Arena?,Did he play in Newcastle?,0.0,0.117647,0.081301,0.724174,0.299865
189028,T5_quac_squad,T5,quac,squad,{'defaults': True},When did the movement that would become The United Methodist Church begin?,What was his religion?,0.0,0.000000,0.040984,0.725274,0.299780
198629,T5_quac_triviaqa,T5,quac,triviaqa,{'defaults': True},Who killed Jean Paul Marat?,What did she do after the war?,0.0,0.000000,0.080645,0.701192,0.299497
...,...,...,...,...,...,...,...,...,...,...,...,...
191587,T5_quac_triviaqa,T5,quac,triviaqa,{'defaults': True},"In 480 BC, Leonidas led his 300 Spartans against the Persian army, led by Xerxes, in which land battle?",Are there any other interesting aspects about this article?,0.0,0.000000,0.023041,0.601840,-0.122759
199891,T5_quac_triviaqa,T5,quac,triviaqa,{'defaults': True},"Brown, American Rainbow and Cutthroat are all types of which fish?",Did he have any other interesting aspects about this article?,0.0,0.000000,0.039062,0.677861,-0.122829
172579,T5_quac_nq,T5,quac,nq,{'defaults': True},when were birth certificates first issued in the united states,Are there any other interesting aspects about this article?,0.0,0.000000,0.000000,0.617690,-0.122929
192874,T5_quac_triviaqa,T5,quac,triviaqa,{'defaults': True},The Bells of Shandon are housed in the Church of St Anne in which Irish city?,Did he have any other interesting aspects about this article?,0.0,0.000000,0.030488,0.643597,-0.140799


In [ ]:
p_quac = quac_low.groupby(["prediction"]).count().sort_values(by="use", ascending=False).reset_index()

p_quac = p_quac.head()

In [ ]:
# Convert to Latex
print(p_quac.to_latex(index=False))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
                                                   prediction &  nickname &  base\_model &  trained\_on &  tested\_on &  hyperparameter &  target &  bleu &  rougeL &  meteor &  bertscore-f1 &  use \\
\midrule
  Are there any other interesting aspects about this article? &      3870 &        3870 &        3870 &       3870 &            3870 &    3870 &  3870 &    3870 &    3870 &          3870 & 3870 \\
                                              What did he do? &      3480 &        3480 &        3480 &       3480 &            3480 &    3480 &  3480 &    3480 &    3480 &          3480 & 3480 \\
                                       Did he win any awards? &      1566 &        1566 &        1566 &       1566 &            1566 &    1566 &  1566 &    1566 &    1566 &          1566 & 1566 \\
                             What did he do after his career? &       878 &         878 &         878 &        878 &             878 &     878 &   878 &     878 

In [ ]:
# Show examples where it looks like a "default" question is being generated
spec_ex = quac_low[
    (quac_low["prediction"] == "What else did he do?")
]

spec_ex = spec_ex[["nickname", "target", "prediction", "bleu", "rougeL", "meteor", "bertscore-f1", "use"]]

spec_ex

,nickname,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
183935,T5_quac_squad,What is the full name of the ASER?,What else did he do?,0.0,0.153846,0.114943,0.721591,0.278790
181007,T5_quac_squad,What was happening to subscriber numbers in other areas of europe?,What else did he do?,0.0,0.125000,0.087719,0.729966,0.252805
182225,T5_quac_squad,What role did Michael Oppenheimer have in the IPCC's reports?,What else did he do?,0.0,0.250000,0.131579,0.716897,0.247044
181242,T5_quac_squad,What is NSFNET,What else did he do?,0.0,0.250000,0.151515,0.676610,0.241931
180335,T5_quac_squad,What famous person congratulated him?,What else did he do?,0.0,0.200000,0.166667,0.725738,0.226569
...,...,...,...,...,...,...,...,...
173237,T5_quac_nq,royal society for the protection of birds number of members,What else did he do?,0.0,0.000000,0.000000,0.521592,-0.059570
182352,T5_quac_squad,What theorem states that the probability that a number n is prime is inversely proportional to its logarithm?,What else did he do?,0.0,0.086957,0.056497,0.618607,-0.072348
182196,T5_quac_squad,The 1970s allowed which network to move in to first place in the ratings?,What else did he do?,0.0,0.000000,0.035461,0.649218,-0.080876
182902,T5_quac_squad,Prevenient grace allowas those tainted by what to make a choice to accept or reject God's salvation in Christ?,What else did he do?,0.0,0.080000,0.076923,0.678948,-0.106613


In [ ]:
gen_q = spec_ex.tail()

gen_q

,nickname,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
173379,T5_quac_nq,in the process of science hypotheses lead most directly to,What did he do?,0.0,0.0,0.0,0.624783,-0.057241
172210,T5_quac_nq,in another world with my smartphone japanese title,What did he do?,0.0,0.0,0.0,0.590820,-0.067950
174106,T5_quac_nq,the joint between a coxal bone of the pelvis and the sacrum,What did he do?,0.0,0.0,0.0,0.559357,-0.068225
171814,T5_quac_nq,is greenland part of europe or north america,What did he do?,0.0,0.0,0.0,0.581262,-0.093530
171946,T5_quac_nq,the reciprocal lattice of an fcc crystal is,What did he do?,0.0,0.0,0.0,0.576945,-0.098762


In [ ]:
# Convert to Latex
print(gen_q.to_latex(index=False))

\begin{tabular}{lllrrrrr}
\toprule
  nickname &                                                      target &      prediction &  bleu &  rougeL &  meteor &  bertscore-f1 &       use \\
\midrule
T5\_quac\_nq &  in the process of science hypotheses lead most directly to & What did he do? &   0.0 &     0.0 &     0.0 &      0.624783 & -0.057241 \\
T5\_quac\_nq &          in another world with my smartphone japanese title & What did he do? &   0.0 &     0.0 &     0.0 &      0.590820 & -0.067950 \\
T5\_quac\_nq & the joint between a coxal bone of the pelvis and the sacrum & What did he do? &   0.0 &     0.0 &     0.0 &      0.559357 & -0.068225 \\
T5\_quac\_nq &                is greenland part of europe or north america & What did he do? &   0.0 &     0.0 &     0.0 &      0.581262 & -0.093530 \\
T5\_quac\_nq &                 the reciprocal lattice of an fcc crystal is & What did he do? &   0.0 &     0.0 &     0.0 &      0.576945 & -0.098762 \\
\bottomrule
\end{tabular}



In [ ]:
# Find predictions with low semantic metric scores when tested on QuAC

quac_low = clean_eval_df[
    (clean_eval_df["trained_on"] == "amalgam")
    & (clean_eval_df["bertscore-f1"] <= 0.73)  # top 25th percentile bertscore
    & (clean_eval_df["use"] <= 0.3)   # bottom 25th percentile use
    & (clean_eval_df["tested_on"] == "quac")
]

quac_low = quac_low.sort_values(by="use", ascending=False)

quac_low

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
119694,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},How long was taking barbiturates?,What did the autopsy show about her stomach?,0.0,0.000000,0.079365,0.707513,0.299898
119134,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},Did he go on to any other notable matches?,Are there any other interesting aspects about this article?,0.0,0.222222,0.255556,0.729418,0.299652
264748,T5_amalgam_quac,T5,amalgam,quac,{'defaults': True},Did they win any awards for these singles,How did the song do on the charts?,0.0,0.125000,0.061728,0.709668,0.299482
261235,T5_amalgam_quac,T5,amalgam,quac,{'defaults': True},did they really retire or was it for a short period?,Are there any other interesting aspects about this article?,0.0,0.000000,0.042373,0.685931,0.299449
120791,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},WHen was his second term?,In what year did Yeltsin undergo emergency heart bypass surgery?,0.0,0.000000,0.076923,0.722384,0.299418
...,...,...,...,...,...,...,...,...,...,...,...,...
118627,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},any intresting information?,When did Chuck Berry return to Chess?,0.0,0.000000,0.113636,0.715174,-0.065768
120979,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},Are there any other interesting aspects about this article?,When did Kanye West become general manager of the Memphis Grizzlies?,0.0,0.000000,0.049020,0.624197,-0.066777
264755,T5_amalgam_quac,T5,amalgam,quac,{'defaults': True},Any others?,When did the album come out?,0.0,0.000000,0.147059,0.695957,-0.081780
120381,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},What is a Portal approach?,Did they release any albums during this time?,0.0,0.000000,0.079365,0.702668,-0.083666


In [ ]:
# Find BART predictions with semantic metric score differences

bart_low = bart_df[
    (bart_df["trained_on"] == "amalgam")
    & (bart_df["bertscore-f1"] <= 0.73)  # top 25th percentile bertscore
    & (bart_df["use"] <= 0.3)   # bottom 25th percentile use
    & (bart_df["tested_on"] == "triviaqa")
    # & ((bart_df["tested_on"] == "quac") | (bart_df["tested_on"] == "triviaqa"))
]

bart_low = bart_low.sort_values(by="use", ascending=False)

bart_low

Look at context for examples above

In [ ]:
prediction_root = "/content/drive/MyDrive/w266 NLP Final Project/Predictions/"
filename = join(prediction_root, "predictions.bart_base_pt_long.amalgam.triviaqa.csv")

csv_df = pd.read_csv(filename)

In [ ]:
csv_df = csv_df[csv_df["target"] == "Who's missing: Inky, Pinky, and Sue"]

csv_df

context  \
3942                                                                                                                                                                             The Ghosts , primarily Blinky , Pinky , Inky and Clyde , are the monsters chasing the player in the Pac-Man franchise . \n \n History \n \n The enemies in Pac-Man are known variously as `` ghosts , '' `` goblins , '' and `` monsters '' . \n Despite the seemingly random nature of the enemies , their movements are strictly deterministic , which players have used to their advantage . In an interview , creator Toru Iwatani stated that he had designed each enemy with its own distinct personality in order to keep the game from becoming impossibly difficult or boring to play . More recently , Iwatani described the enemy behaviors in more detail at the 2011 Game Developers Conference . He stated that the red enemy chases Pac-Man , and the pink and blue enemies try to position themselves in front of Pac-Man 's mouth . Although he claimed that the orange enemy 's behavior is random , the game 's code reveals that it actually chases Pac-Man most of the time , but also moves toward the lower-left corner of the maze when it gets too close to Pac-Man . \n \n Cartoons \n \n In the Pac-Man cartoon , with the male Ghost Monsters wearing various styles of hats and partnered with a new ghost named Sue ( who was purple and wore earrings ) , the Ghost Monsters work for Mezmaron who assigns them the job of finding the Power Pellet Forest . Just as Pac-Man could `` chomp '' the Ghost Monsters to devour their `` suits '' after gulping down a power pellet and thus turning the Ghost Monsters blue — with their disembodied `` eyes '' floating back to Mezmaron 's lair — the Ghost Monsters themselves , whenever Pac-Man had n't been able to consume a power pellet for a time , could likewise `` chomp '' Pac-Man to defeat him . The episodes `` Chomp-Out at the O.K . Corral '' and `` The Greatest Show in Pacland '' featured the Ghost Monsters ' 3-year-old cousin named Dinky who was bratty , which greatly annoyed all the male ghost monsters . \n \n In the 2013 TV series Pac-Man and the Ghostly Adventures , the four Ghosts come from the Netherworld . Though they are ruled by Lord Betrayus , they are actually good-natured spirits and often supply Pac-Man with information about Lord Betrayus ' plots , while ensuring Betrayus does n't catch them in the act . It is also suggested that they could be reunited with their bodies and brought back to life , though their 'living ' forms are unknown . There were also some Ghosts that were exclusive to the TV series like Cyclops Ghosts ( a race of heavyset , horned Ghosts with one eye ) , Fire Ghosts ( a race of orange Ghosts who can emit fire from their body ) , Tentacle Ghosts ( a race of 4-eyed purple-black Ghosts who look similar to jellyfish ) , Guardian Ghosts ( a race of large Ghosts who guard the Netherworld ) , and Aqua Ghosts ( a race of light blue Ghosts with fins on their head ) . \n \n Known Ghosts \n \n Below is the description of each Ghost . \n \n Blinky \n \n Blinky is a red ghost who , in the original arcade game , follows behind Pac-man . He is considered the leader of the ghosts . \n \n In the Pac-Man cartoon , Blinky ( voiced by Chuck McCann ) is slow-witted and cowardly with grammar problems . \n \n In Pac-Man and the Ghostly Adventures , Pac-Man and the Ghostly Adventures 2 , & Mappy : The Movie , Blinky ( voiced by Ian James Corlett in the TV series and by Lucien Dodge in the video game ) is the default leader of the Ghost Gang Family and tends to help the winning side . \n \n Blinky receives a speed boost after a number of pac-pellets have been cleared . This mode has been informally referred to as `` Cruise Elroy '' . \n \n Pinky \n \n Pinky is a pink ghost who , in the original arcade game , positions him/herself in front of Pac-man . \n \n In the Pac-Man cartoon , Pinky ( voiced by Chuck McCann ) is depicted as m

### Set Up Probe

In [ ]:
class Probe:
    """
    A class used to represent an Awesome Puppies Question Generation Probe
    ...

    Attributes
    ----------
    models : dict
        a dictionary of cached models
    tokenizers : dict
        a dictionary of cached tokenizers

    Methods
    -------
    predict(context, answer, base_model='bart', training_dataset='amalgam',
            num_beams=4, early_stopping=True, no_repeat_ngram_size=3,
            maximum_input_length=1024, maximum_target_length=50)

        returns a prediction string

    retrieve_tokenizer (base_model='bart')

        returns and caches a tokenizer

    retrieve_model(base_model='bart', training_dataset='amalgam')

        returns and caches a model
    """

    project_root = "/content/drive/MyDrive/w266 NLP Final Project/"
    #    project_root = "/home/localadmin/Documents/w266_NLP_Final_Project/"
    model_root = project_root + "Models/"

    def __init__(self):
        self.models = {}
        self.tokenizers = {}

    def predict(
        self,
        context,
        answer,
        base_model="bart",
        training_dataset="amalgam",
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3,
        maximum_input_length=1024,
        maximum_target_length=50,
    ):

        tokenizer = self.retrieve_tokenizer(base_model)
        model = self.retrieve_model(base_model, training_dataset)

        if base_model == "bart":
            prompt_string = f"{answer}  {context}"
        elif base_model == "T5":
            prompt_string = f"generate question: answer: {answer} context: {context}"
        else:
            raise ValueError("invalid base model")

        inputs = tokenizer(
            prompt_string,
            return_tensors="pt",
            max_length=maximum_input_length,
            truncation=True,
            padding=True,
        )
        output_ids = model.generate(
            inputs["input_ids"].cuda(),
            max_length=maximum_target_length,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=early_stopping,
        )
        prediction = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        return prediction

    def retrieve_tokenizer(self, base_model="T5"):
        tokenizer = self.tokenizers.get(base_model)
        if tokenizer:
            return tokenizer

        if base_model == "bart":
            tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
        elif base_model == "T5":
            tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")
        else:
            raise ValueError("invalid base model")

        self.tokenizers[base_model] = tokenizer
        return tokenizer

    def retrieve_model(self, base_model="bart", training_dataset="amalgam"):
        model_tuple = (base_model, training_dataset)
        model = self.models.get(model_tuple)
        if model:
            return model

        model_dir = f"{self.model_root}{base_model}_base_pt_long.{training_dataset}"

        if base_model == "bart":
            print(f"Loading: {model_dir}")
            model = BartForConditionalGeneration.from_pretrained(model_dir)
        elif base_model == "T5":
            print(f"Loading: {model_dir}")
            model = T5ForConditionalGeneration.from_pretrained(model_dir)
        else:
            raise ValueError("invalid base model")

        model.to(torch.device("cuda:0"))
        self.models[model_tuple] = model
        return model

In [ ]:
def panel(
    probe,
    context,
    answers,
    base_models,
    training_datasets,
    num_beams=4,
    early_stopping=True,
    no_repeat_ngram_size=3,
    maximum_input_length=1024,
    maximum_target_length=50,
):
    result = [
        {
            "Base Model": base_model,
            "Training Dataset": training_dataset,
            "Answer": answer,
            "Prediction": "".join(
                probe.predict(
                    context,
                    answer,
                    base_model=base_model,
                    training_dataset=training_dataset,
                    num_beams=num_beams,
                    no_repeat_ngram_size=no_repeat_ngram_size,
                    early_stopping=early_stopping,
                    maximum_input_length=maximum_input_length,
                    maximum_target_length=maximum_target_length,
                )
            ),
        }
        for answer in answers
        for base_model in base_models
        for training_dataset in training_datasets
    ]
    replacements = {
        "bart": "BART",
        "amalgam": "Shuffled Blended",
        "nq": "NQ",
        "quac": "QuAC",
        "squad": "SQuAD",
        "triviaqa": "TriviaQA",
    }
    return pd.DataFrame(result).replace(replacements)

Analysis with Probe

In [ ]:
p = Probe()

In [ ]:
df = panel(
    p,
    fossie_had_a_little_lamb,
    ["Fossie", "little", "white"],
    ["T5", "bart"],
    ["amalgam", "squad"],
)
df

## Format code to PEP 8 Standards

### Steps

*   Install:

In [ ]:
!pip install black[jupyter]


*   To format your code run:

In [ ]:
!black /content/drive/MyDrive/'Colab Notebooks'/results_text_evaluation.ipynb


*   Don't save your notebook, hit F5 (Command + r) to refresh the page
*   Voila!
*   Now save!